In [4]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Literal
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage,HumanMessage,BaseMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool

import requests
import random

In [5]:
load_dotenv()  # Load .env from the project root

True

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=os.getenv("GOOGLE_API_KEY"))

In [21]:
# tools 
search_tool = DuckDuckGoSearchRun()

@tool
def calculator(first_num:float ,second_num:float ,operation:str) -> dict:
    """Performs basic arithmetic operations and returns a dict with the result or error."""
    if operation == 'add':
        result = first_num + second_num
    elif operation == 'subtract':
        result = first_num - second_num
    elif operation == 'multiply':
        result = first_num * second_num
    elif operation == 'divide':
        if second_num == 0:
            return {"error": "Cannot divide by zero."}
        result = first_num / second_num
    else:
        return {"error": "Invalid operation. Choose from 'add', 'subtract', 'multiply', 'divide'."}
    return {"result": result}


@tool
def get_stock_price(ticker_symbol: str) -> dict:
    """Fetches the current stock price for the given ticker symbol."""
    api_key = os.getenv("ALPHAVANTAGE_API_KEY")
    url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={ticker_symbol}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    
    try:
        price = data["Global Quote"]["05. price"]
        return {"ticker": ticker_symbol, "price": price}
    except KeyError:
        return {"error": "Invalid ticker symbol or data not available."}

In [22]:
# tools litst 
tools=[search_tool,calculator,get_stock_price]

# making llm aware of tools
llm_with_tools=llm.bind_tools(tools)

In [23]:
# state 
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [24]:
# grpah node
def chat_node (state:ChatState):
  """LLm node that may answer or use tools"""
  messages=state['messages']
  response=llm_with_tools.invoke(messages)
  return {"messages":[response]}

In [25]:
# workflow 
graph = StateGraph(ChatState)

# Define nodes
graph.add_node("chat_node", chat_node)

# instantiate a ToolNode using the existing 'tools' list and add it to the graph
tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

# Define edges
graph.add_edge(START, "chat_node")
graph.add_conditional_edges("chat_node", tools_condition)
graph.add_edge("tools", "chat_node")

In [29]:
chatbot=graph.compile()

In [30]:
chatbot

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [31]:

# Regular chat
out = chatbot.invoke({"messages": [HumanMessage(content="Hello!")]})

print(out["messages"][-1].content)

Hello! I'm a an AI assistant. I can answer questions, perform calculations, and fetch current stock prices. What can I do for you today? 


In [32]:

# Chat requiring tool
out = chatbot.invoke({"messages": [HumanMessage(content="What is 2*3?")]})
print(out["messages"][-1].content)

2 multiplied by 3 is 6.


In [33]:

# Chat requiring tool
out = chatbot.invoke({"messages": [HumanMessage(content="What is the stock price of apple")]})
print(out["messages"][-1].content)

To provide you with the Apple stock price, I need its ticker symbol. Could you please provide it?


In [34]:

# Chat requiring tool
out = chatbot.invoke({"messages": [HumanMessage(content="First find out the stock price of Apple using get stock price tool then use the calculator tool to find out how much will it take to purchase 50 shares?")]})
print(out["messages"][-1].content)

[{'type': 'text', 'text': 'The cost to purchase 50 shares of Apple (AAPL) is 13683.5.', 'extras': {'signature': 'CvgBAXLI2nwMS2A35DP0ne22bzGiLaV3yoLGJO8U4eLz83AJInxf3XtXNOYgv4wQcj6aUpjT6+CB1Ku9gQA2TkAW9Ht9gf5+Cz/+baHxtFvI9LyoQDP51hddK7SiU+zUpWnO3PtQYfVZHzpv6vEmVzFkFI5thyIymBaLtb5EZ6q/JcJD6yGz3/M5fhTRKbJYqjoQqH9+2HTKgMzAMXDWlgqEq79c1dfEzIxgrts3KOp7I7PiEvnuCr144YeJTmXlvBzFMTGrWX/DTlDXmk7caHWzUqGwCUQLPV0St50aMlQQUvMOrcbmDjDcaEZ3OtCiRbbDqMPnkNAntpY='}}]
